In [1]:
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [32]:
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import pandas as pd
import numpy as np
import random

In [4]:
random.seed(0)

In [5]:
import matplotlib.pyplot as plt

In [6]:
import pickle


In [7]:
def train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after):
    random.seed(0)
    test_set = np.concatenate(((np.random.choice(np.array(Asthmatic_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Asthmatic_Female), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Female), replace=False, size=(7)))))
    train_set = np.concatenate((np.array(Asthmatic_Male),
                                np.array(Asthmatic_Female),
                                np.array(Healthy_Male),
                                np.array(Healthy_Female)))
    train_set = np.setdiff1d(train_set,test_set)
#     np.array(before_after).shape
    train_set = np.concatenate((train_set, np.array(before_after)))
    return train_set, test_set

In [114]:
Breath_dataframe = pd.read_csv("./breath_csv/Breath.csv")
Breath_1_dataframe = pd.read_csv("./breath_csv/Breath_1.csv")
Breath_2_dataframe = pd.read_csv("./breath_csv/Breath_2.csv")
Breath_3_dataframe = pd.read_csv("./breath_csv/Breath_3.csv")

### DATA_STATS

In [9]:
print(Breath_1_dataframe.shape)
print(Breath_2_dataframe.shape)
print(Breath_3_dataframe.shape)

(2306, 74)
(2306, 74)
(2306, 74)


In [26]:
Breath_1_dataframe

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,-7.518903,-10.990897,-9.604292,3.563389,-0.873071,-7.521612,-6.878887,-27.168114,0.654916,0.897164,...,5.748509,9.964848,10.479784,27.903249,7.502630,7.340091,12.249179,9.234270,1.0,2030_P_Asthma_M_after_TBD_Y.json
1,-5.193252,-18.613230,-11.146862,-5.760457,-3.999578,-13.968749,-6.796815,-17.419335,-11.429387,6.806179,...,10.009527,15.687760,11.113335,20.091881,14.834731,10.605124,9.670656,9.676993,1.0,2030_P_Asthma_M_after_TBD_Y.json
2,-6.096903,-10.911015,-2.237536,12.137149,-9.991736,-11.228307,-6.328688,-22.451297,-8.170443,-19.182946,...,12.199509,14.145921,11.726745,24.582509,11.254667,24.903076,12.163624,9.825561,1.0,2030_P_Asthma_M_after_TBD_Y.json
3,-6.935391,-8.036528,-8.586313,7.782491,-5.311475,-2.698300,-7.505101,-27.025777,-6.795934,-3.166907,...,7.148386,7.174043,11.459255,27.840742,9.486919,8.353772,13.430368,8.722226,1.0,2030_P_Asthma_M_after_TBD_Y.json
4,-7.587186,-16.303047,-9.174580,-4.558920,-4.485484,0.892777,-14.394063,-17.377047,-8.319141,3.556375,...,7.439801,6.795588,16.693376,18.482805,11.581451,8.897151,10.264586,10.755023,1.0,2030_P_Asthma_M_after_TBD_Y.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,-4.320324,-9.879842,-13.928946,-13.810120,5.398152,2.704029,14.464028,-7.172482,9.852755,13.422723,...,9.935237,8.270486,16.887481,21.006640,13.090739,18.377831,12.821605,16.855565,0.0,1306_C_Na_M_Na_Na_N.json
2302,-3.303135,-24.250914,-30.332555,-4.667135,7.353563,-2.535968,7.425988,-6.300057,-7.821027,10.426692,...,9.628649,7.781631,10.826065,10.777435,11.931527,13.236125,8.034495,23.847981,0.0,1306_C_Na_M_Na_Na_N.json
2303,-3.627535,-25.670246,-27.080315,-3.629266,7.596613,-1.875681,7.555803,-12.493904,-2.613590,11.066152,...,10.227850,8.383238,11.042578,14.996092,7.796741,13.351076,9.287066,18.485545,0.0,1306_C_Na_M_Na_Na_N.json
2304,-3.413044,-29.601389,-29.711291,3.127272,5.475462,-7.608470,17.674903,-13.802974,-7.210363,17.010800,...,8.014320,10.995430,19.519234,15.823024,10.469216,19.236726,8.662705,19.157752,0.0,1306_C_Na_M_Na_Na_N.json


In [11]:
Breath_2_dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,-5.819893,-0.807137,-5.621214,5.254770,-3.315850,5.227212,2.699075,-10.106676,-2.065468,3.210382,...,7.525397,13.206269,8.542131,12.448360,11.166359,14.285654,9.299346,14.721490,1.0,2030_P_Asthma_M_after_TBD_Y.json
1,-4.239213,-5.541188,-4.688828,-0.346142,-6.847378,-2.986326,-0.986394,-12.044204,-6.977862,1.198398,...,12.489347,20.389885,9.178060,16.179534,22.209952,15.749400,12.588714,17.929530,1.0,2030_P_Asthma_M_after_TBD_Y.json
2,-5.354658,-5.668823,-2.405999,10.432677,-7.490145,-3.756268,4.655690,-25.750033,-1.074364,-7.176282,...,10.019886,11.460915,10.751927,26.900385,8.573363,14.580283,11.540474,11.574759,1.0,2030_P_Asthma_M_after_TBD_Y.json
3,-5.092278,-2.529580,-9.688043,9.188193,-12.220343,-6.892942,3.612338,-20.445863,4.298290,-0.158647,...,16.611896,13.117266,10.425394,25.338480,12.605503,16.826520,15.302202,16.827541,1.0,2030_P_Asthma_M_after_TBD_Y.json
4,-5.227036,-2.247611,-6.643047,3.928853,-4.373642,5.126797,0.458337,-8.708336,0.721195,12.353288,...,6.839993,9.704536,6.540969,11.134982,11.737423,17.516899,8.658353,22.654186,1.0,2030_P_Asthma_M_after_TBD_Y.json


In [11]:
labels = Breath_1_dataframe.iloc[:,73]

In [22]:
print(f"Number of Subjects: {np.unique(labels.values).shape[0]}")

Number of Subjects: 258


In [14]:
sound_dataframe = Breath_1_dataframe

In [16]:
Total = np.unique(sound_dataframe.iloc[:,sound_dataframe.shape[1]-1])
name = []
for i in Total:
    name.append(i[:4])

unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
for file in unique:
    if file.find('sthma') !=-1:
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)

In [26]:
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")

Asthmatic_male = 78
Asthmatic_female = 55
Healthy_Male = 55
Healthy_female = 50
Before_after = 20


In [12]:
# train_test Csv
def train_test_csv(sound_dataframe):
    Total = np.unique(sound_dataframe.iloc[:,sound_dataframe.shape[1]-1])
    name = []
    for i in Total:
        name.append(i[:4])

    unique = []
    before_after = []
    for i in Total:
        if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
            unique.append(i)
        else:
            before_after.append(i) 

    Asthmatic_Female = []
    Asthmatic_Male = []
    Healthy_Male = []
    Healthy_Female = []
    for file in unique:
        if file.find('sthma') !=-1:
            if file.find("_M_")!=-1:
                Asthmatic_Male.append(file)
            if file.find("_F_")!=-1:
                Asthmatic_Female.append(file)
        if file.find("_C_")!=-1:
            if file.find("_M_")!=-1:
                Healthy_Male.append(file)
            if file.find("_F_")!=-1:
                Healthy_Female.append(file)

                

    Train, Test = train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after)     

    test_dataframe = pd.DataFrame()
    for i, line in enumerate(Test):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[i]))]
        test_dataframe = pd.DataFrame.append(test_dataframe,A)
        
        
    train_dataframe = pd.DataFrame()
    for i, line in enumerate(Train):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[i]))]
        train_dataframe = pd.DataFrame.append(train_dataframe,A) 
        
        
    
    return train_dataframe, test_dataframe

In [94]:
#Machine Learning Algorithm (MLA) Selection and Initialization
def MLA_selection(sound_dataframe, sound):    
    MLA = [
        #Ensemble Methods
#         ensemble.AdaBoostClassifier(),
#         ensemble.BaggingClassifier(),
#         ensemble.ExtraTreesClassifier(),
        ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier(),

        #Gaussian Processes
#         gaussian_process.GaussianProcessClassifier(),

        #GLM
#         linear_model.LogisticRegressionCV(),
#         linear_model.PassiveAggressiveClassifier(),
#         linear_model.RidgeClassifierCV(),
#         linear_model.SGDClassifier(),
#         linear_model.Perceptron(),

        #Navies Bayes
#         naive_bayes.BernoulliNB(),
#         naive_bayes.GaussianNB(),

        #Nearest Neighbor
#         neighbors.KNeighborsClassifier(),

        #SVM
        svm.SVC(probability=True),
#         svm.NuSVC(probability=True),
        svm.LinearSVC(),

        #Trees    
#         tree.DecisionTreeClassifier(),
# #         tree.ExtraTreeClassifier(),

        #Discriminant Analysis
#         discriminant_analysis.LinearDiscriminantAnalysis(),
#         discriminant_analysis.QuadraticDiscriminantAnalysis(),


        XGBClassifier()
#         XGBClassifier()
        ]


    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy','MLA Train Accuracy Mean', 'MLA Test Accuracy', 'MLA Test Accuracy Mean','MLA Test Accuracy Std' ]
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        test = []
        train = []
        for i in tqdm(range(0,10)):
            train_csv, test_csv = pd.read_csv('./breathe_set_csv/train'+sound+str(i)+'.csv'), pd.read_csv('./breathe_set_csv/test'+sound+str(i)+'.csv')
            X_train = train_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_train = train_csv.iloc[:,sound_dataframe.shape[1]-2]
            X_test = test_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_test = test_csv.iloc[:,sound_dataframe.shape[1]-2]
            #Scaling
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = pd.DataFrame(X_train)
            X_test = pd.DataFrame(X_test)

            alg.fit(X_train,y_train)
            test.append(metrics.accuracy_score(y_test,alg.predict(X_test)))

            train.append(metrics.accuracy_score(y_train,alg.predict(X_train)))
            
#             if alg.__class__.__name__ == 'XGBClassifier':
#                 filename = './CMN/XGB_model'+ str(i) + '.model'
#                 pickle.dump(alg, open(filename, 'wb'))
#                 train_csv.to_csv('./CMN/Train_XGB'+str(i)+'.csv', index=False)
                
        
        MLA_compare.loc[row_index, 'MLA Train Accuracy'] = train
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = np.mean(train)
        MLA_compare.loc[row_index, 'MLA Test Accuracy'] = test
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = np.mean(test)
        MLA_compare.loc[row_index, 'MLA Test Accuracy Std'] = np.std(test)


        row_index+=1

    
    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    
    return MLA_compare

In [34]:
# Making Sets
def make_and_save_sets(sound_dataframe, sound):
    for i in range(0,10):
        train_csv, test_csv = train_test_csv(sound_dataframe)
        train_csv.to_csv('./breathe_set_csv/train'+sound+str(i)+'.csv', index=False)
        test_csv.to_csv('./breathe_set_csv/test'+sound+str(i)+'.csv', index=False)

### Breath

In [46]:
make_and_save_sets(Breath_dataframe, 'breath')

In [60]:
MLA_breathe = MLA_selection(Breath_dataframe, 'breath')

  0%|          | 0/10 [00:00<?, ?it/s]

[14:53:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:07,  1.22it/s]

[14:53:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:01<00:05,  1.43it/s]

...

 80%|████████  | 8/10 [00:04<00:01,  1.68it/s]

[14:53:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:05<00:00,  1.70it/s]

[14:53:10] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:06<00:00,  1.61it/s]


In [61]:
MLA_breathe

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9602713178294574, 0.9722762645914397, 0.966...",0.962408,"[0.7148760330578512, 0.776, 0.7900763358778626...",0.76422,0.026619
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7355371900826446, 0.768, 0.7786259541984732...",0.764178,0.0298449
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6983471074380165, 0.752, 0.7824427480916031...",0.750189,0.0348382
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9563953488372093, 0.958171206225681, 0.9530...",0.956912,"[0.6487603305785123, 0.748, 0.7519083969465649...",0.73822,0.0513505
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8410852713178295, 0.8321984435797666, 0.836...",0.83542,"[0.7066115702479339, 0.812, 0.683206106870229,...",0.731677,0.0448544


### Breath_1

In [35]:
make_and_save_sets(Breath_1_dataframe, 'breath_1')

In [62]:
MLA_breathe_1 = MLA_selection(Breath_1_dataframe, 'breath_1')

  0%|          | 0/10 [00:00<?, ?it/s]

[14:54:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:06,  1.41it/s]

[14:54:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:01<00:05,  1.50it/s]

...

 80%|████████  | 8/10 [00:05<00:01,  1.46it/s]

[14:54:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:06<00:00,  1.45it/s]

[14:54:47] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:06<00:00,  1.50it/s]


In [63]:
MLA_breathe_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9115216334467672, 0.9064642507345739, 0.909...",0.911477,"[0.6546184738955824, 0.7196969696969697, 0.768...",0.715953,0.041384
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6465863453815262, 0.6818181818181818, 0.756...",0.708228,0.0435349
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6746987951807228, 0.6590909090909091, 0.741...",0.698926,0.0349025
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.7894992707826932, 0.7894221351616063, 0.775...",0.78429,"[0.6305220883534136, 0.696969696969697, 0.7529...",0.695524,0.0490389
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9261059795819154, 0.9280117531831538, 0.919...",0.92486,"[0.606425702811245, 0.6856060606060606, 0.7450...",0.682834,0.042769


### Breath_2

In [38]:
make_and_save_sets(Breath_2_dataframe, 'breath_2')

In [65]:
MLA_breathe_2 = MLA_selection(Breath_2_dataframe, 'breath_2')

  0%|          | 0/10 [00:00<?, ?it/s]

[14:56:12] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:05,  1.55it/s]

[14:56:12] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:01<00:05,  1.56it/s]

...

 80%|████████  | 8/10 [00:05<00:01,  1.53it/s]

[14:56:17] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:05<00:00,  1.57it/s]

[14:56:17] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:06<00:00,  1.55it/s]


In [66]:
MLA_breathe_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7366255144032922, 0.8607594936708861, 0.65,...",0.735733,0.0737277
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9510421715947649, 0.9453842435959401, 0.953...",0.949815,"[0.7695473251028807, 0.8059071729957806, 0.692...",0.733199,0.0693068
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9360155113911779, 0.9376510391493476, 0.938...",0.939837,"[0.7654320987654321, 0.7848101265822784, 0.630...",0.727105,0.0665275
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8221037324285022, 0.8158530691155147, 0.835...",0.827117,"[0.7613168724279835, 0.7805907172995781, 0.611...",0.723726,0.058919
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7695473251028807, 0.8270042194092827, 0.611...",0.72334,0.0811724


## Breath_3

In [41]:
make_and_save_sets(Breath_3_dataframe, 'breath_3')

In [67]:
MLA_breathe_3 = MLA_selection(Breath_3_dataframe, 'breath_3')

  0%|          | 0/10 [00:00<?, ?it/s]

[14:57:42] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:06,  1.41it/s]

[14:57:43] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:01<00:05,  1.46it/s]

...

 80%|████████  | 8/10 [00:05<00:01,  1.53it/s]

[14:57:47] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:05<00:00,  1.53it/s]

[14:57:48] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:06<00:00,  1.52it/s]


In [68]:
MLA_breathe_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8242597898758357, 0.8243831640058055, 0.816...",0.816199,"[0.6933962264150944, 0.7071129707112971, 0.756...",0.760806,0.0474357
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9235912129894938, 0.9216255442670537, 0.923...",0.919384,"[0.6839622641509434, 0.7405857740585774, 0.741...",0.758893,0.0444663
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6509433962264151, 0.7238493723849372, 0.745...",0.747235,0.0507716
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6839622641509434, 0.7280334728033473, 0.749...",0.739164,0.0493696
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9283667621776505, 0.9269472665699081, 0.923...",0.92613,"[0.6556603773584906, 0.698744769874477, 0.7414...",0.735363,0.0566528


In [ ]:
pd.read_csv("./breathe_set_csv/t")

## Experiment 2

In [79]:
Breath_dataframe_0 = pd.read_csv("./breath_csv/Breath_0.csv")
Breath_dataframe_5 = pd.read_csv("./breath_csv/Breath_5.csv")
Breath_dataframe_10 = pd.read_csv("./breath_csv/Breath_10.csv")
Breath_dataframe_15 = pd.read_csv("./breath_csv/Breath_15.csv")
Breath_dataframe_20 = pd.read_csv("./breath_csv/Breath_20.csv")
Breath_dataframe_25 = pd.read_csv("./breath_csv/Breath_25.csv")
Breath_dataframe_30 = pd.read_csv("./breath_csv/Breath_30.csv")
Breath_dataframe_35 = pd.read_csv("./breath_csv/Breath_35.csv")
Breath_dataframe_40 = pd.read_csv("./breath_csv/Breath_40.csv")
Breath_dataframe_45 = pd.read_csv("./breath_csv/Breath_45.csv")
Breath_dataframe_50 = pd.read_csv("./breath_csv/Breath_50.csv")


In [91]:
make_and_save_sets(Breath_dataframe_0, 'Breath_0')
make_and_save_sets(Breath_dataframe_5, 'Breath_5')
make_and_save_sets(Breath_dataframe_10, 'Breath_10')
make_and_save_sets(Breath_dataframe_15, 'Breath_15')
make_and_save_sets(Breath_dataframe_20, 'Breath_20')
make_and_save_sets(Breath_dataframe_25, 'Breath_25')
make_and_save_sets(Breath_dataframe_30, 'Breath_30')
make_and_save_sets(Breath_dataframe_35, 'Breath_35')
make_and_save_sets(Breath_dataframe_40, 'Breath_40')
make_and_save_sets(Breath_dataframe_45, 'Breath_45')
make_and_save_sets(Breath_dataframe_50, 'Breath_50')

In [95]:
MLA_breathe_0 = MLA_selection(Breath_dataframe_0, 'Breath_0')
MLA_breathe_5 = MLA_selection(Breath_dataframe_5, 'Breath_5')
MLA_breathe_10 = MLA_selection(Breath_dataframe_10, 'Breath_10')
MLA_breathe_15 = MLA_selection(Breath_dataframe_15, 'Breath_15')
MLA_breathe_20 = MLA_selection(Breath_dataframe_20, 'Breath_20')
MLA_breathe_25 = MLA_selection(Breath_dataframe_25, 'Breath_25')
MLA_breathe_30 = MLA_selection(Breath_dataframe_30, 'Breath_30')
MLA_breathe_35 = MLA_selection(Breath_dataframe_35, 'Breath_35')
MLA_breathe_40 = MLA_selection(Breath_dataframe_40, 'Breath_40')
MLA_breathe_45 = MLA_selection(Breath_dataframe_45, 'Breath_45')
MLA_breathe_50 = MLA_selection(Breath_dataframe_50, 'Breath_50')

  0%|          | 0/10 [00:00<?, ?it/s]

[15:50:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:01<00:14,  1.63s/it]

[15:50:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:02<00:09,  1.24s/it]

...

 80%|████████  | 8/10 [00:07<00:01,  1.10it/s]

[16:04:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:08<00:00,  1.10it/s]

[16:04:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:09<00:00,  1.10it/s]


In [96]:
MLA_breathe_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7209302325581395, 0.7272727272727273, 0.727...",0.741458,0.0483478
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.923828125, 0.9190361445783133, 0.9213592233...",0.921767,"[0.7093023255813954, 0.7229437229437229, 0.739...",0.73204,0.0449479
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.95361328125, 0.9450602409638554, 0.94174757...",0.9489,"[0.6782945736434108, 0.6926406926406926, 0.707...",0.721338,0.0506918
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.80419921875, 0.8043373493975904, 0.81456310...",0.804877,"[0.7170542635658915, 0.7142857142857143, 0.695...",0.717376,0.0364233
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6821705426356589, 0.696969696969697, 0.6951...",0.715892,0.0484671


In [101]:
MLA_breathe_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7840909090909091, 0.7113821138211383, 0.722...",0.760624,0.0406329
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9461312438785504, 0.9548543689320388, 0.953...",0.950927,"[0.7765151515151515, 0.7154471544715447, 0.737...",0.751428,0.058931
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9201762977473066, 0.920873786407767, 0.9246...",0.921471,"[0.75, 0.7154471544715447, 0.7376425855513308,...",0.748415,0.0448535
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7651515151515151, 0.7357723577235772, 0.733...",0.747933,0.0517252
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.806562193927522, 0.8058252427184466, 0.8154...",0.804274,"[0.75, 0.7195121951219512, 0.6920152091254753,...",0.736049,0.0575335


In [102]:
MLA_breathe_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7235772357723578, 0.757085020242915, 0.6909...",0.760154,0.0543676
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6991869918699187, 0.7692307692307693, 0.699...",0.756607,0.0684246
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9286407766990291, 0.9281204468188441, 0.921...",0.925257,"[0.6910569105691057, 0.7408906882591093, 0.716...",0.752931,0.0543204
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9495145631067962, 0.9407479358912093, 0.942...",0.945165,"[0.7154471544715447, 0.7206477732793523, 0.660...",0.747687,0.0614878
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8058252427184466, 0.8105876639145216, 0.803...",0.804142,"[0.7195121951219512, 0.631578947368421, 0.6652...",0.723454,0.0650697


In [103]:
MLA_breathe_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9285367039377734, 0.9390838206627681, 0.930...",0.932672,"[0.7429718875502008, 0.7165354330708661, 0.771...",0.7387,0.049778
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7269076305220884, 0.6771653543307087, 0.758...",0.731812,0.0581954
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7068273092369478, 0.7244094488188977, 0.766...",0.731297,0.0533388
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9363150218765192, 0.9507797270955166, 0.940...",0.943725,"[0.6907630522088354, 0.7007874015748031, 0.741...",0.720166,0.0626609
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8055420515313564, 0.800682261208577, 0.8053...",0.810197,"[0.7188755020080321, 0.7165354330708661, 0.665...",0.698275,0.0628996


In [104]:
MLA_breathe_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9473684210526315, 0.944417593040116, 0.9482...",0.946982,"[0.673992673992674, 0.8312236286919831, 0.7968...",0.759149,0.0657523
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6776556776556777, 0.8185654008438819, 0.785...",0.75592,0.0569811
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6886446886446886, 0.8227848101265823, 0.769...",0.751651,0.0603728
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.94146581406788, 0.9371677138714355, 0.93512...",0.937018,"[0.652014652014652, 0.8396624472573839, 0.7539...",0.746479,0.0663379
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8298081652729956, 0.8139197680038666, 0.819...",0.820439,"[0.6703296703296703, 0.8227848101265823, 0.757...",0.733895,0.0629892


In [105]:
MLA_breathe_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8370192307692308, 0.829126213592233, 0.8250...",0.823542,"[0.6548672566371682, 0.7154471544715447, 0.734...",0.750532,0.0513305
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6902654867256637, 0.7682926829268293, 0.7, ...",0.748463,0.0404982
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9403846153846154, 0.9412621359223301, 0.939...",0.938902,"[0.6991150442477876, 0.7682926829268293, 0.738...",0.74813,0.0369506
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9485576923076923, 0.9441747572815534, 0.954...",0.947423,"[0.6902654867256637, 0.7845528455284553, 0.7, ...",0.748,0.0477986
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6991150442477876, 0.7560975609756098, 0.726...",0.741226,0.0429285


In [108]:
MLA_breathe_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9488304093567251, 0.9423170140571983, 0.945...",0.945284,"[0.6732283464566929, 0.7983539094650206, 0.766...",0.771667,0.0568199
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6850393700787402, 0.7530864197530864, 0.766...",0.764239,0.0408503
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.6889763779527559, 0.8024691358024691, 0.770...",0.763884,0.0479508
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9444444444444444, 0.9384391662627242, 0.941...",0.938957,"[0.6811023622047244, 0.7736625514403292, 0.737...",0.75844,0.0487334
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8245614035087719, 0.8099854580707707, 0.825...",0.820978,"[0.7047244094488189, 0.7777777777777778, 0.737...",0.742551,0.0484762


In [109]:
MLA_breathe_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8024193548387096, 0.7154471544715447, 0.732...",0.761618,0.0438694
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9407191448007775, 0.9422330097087379, 0.942...",0.94081,"[0.7540322580645161, 0.7479674796747967, 0.684...",0.751841,0.0345117
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9412050534499514, 0.9451456310679611, 0.943...",0.944605,"[0.7701612903225806, 0.7195121951219512, 0.748...",0.749764,0.0487241
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7701612903225806, 0.7032520325203252, 0.688...",0.748593,0.0476399
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8104956268221575, 0.8310679611650486, 0.819...",0.819893,"[0.7540322580645161, 0.7357723577235772, 0.748...",0.738059,0.0360477


In [110]:
MLA_breathe_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9398642095053347, 0.9458272327964861, 0.953...",0.947545,"[0.819672131147541, 0.8132295719844358, 0.7489...",0.763443,0.0407363
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8319672131147541, 0.8093385214007782, 0.736...",0.754622,0.042072
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9398642095053347, 0.9443631039531479, 0.946...",0.942294,"[0.8032786885245902, 0.7782101167315175, 0.736...",0.750773,0.0295168
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7704918032786885, 0.8171206225680934, 0.740...",0.749178,0.0399472
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8118331716779825, 0.8042947779404588, 0.815...",0.813461,"[0.7745901639344263, 0.8015564202334631, 0.773...",0.734855,0.0521324


In [111]:
MLA_breathe_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8307086614173228, 0.823045267489712, 0.8280...",0.755789,0.0661308
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7834645669291339, 0.8106995884773662, 0.828...",0.747783,0.0713838
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9429824561403509, 0.9365002423654871, 0.940...",0.94218,"[0.7992125984251969, 0.8271604938271605, 0.796...",0.741859,0.0680587
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.7992202729044834, 0.8109549200193893, 0.802...",0.815734,"[0.8346456692913385, 0.7901234567901234, 0.846...",0.733167,0.0916071
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9381091617933723, 0.9389238972370334, 0.943...",0.943719,"[0.8070866141732284, 0.7942386831275721, 0.814...",0.725076,0.0730134


In [112]:
MLA_breathe_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8685258964143426, 0.6638297872340425, 0.795...",0.724131,0.0661764
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8844621513944223, 0.6893617021276596, 0.806...",0.721735,0.0740567
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9343065693430657, 0.9420569773056494, 0.940...",0.942971,"[0.8804780876494024, 0.6553191489361702, 0.791...",0.721247,0.0697468
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8, 0.8338966682761951, 0.8090551181102362, ...",0.819019,"[0.8725099601593626, 0.6340425531914894, 0.813...",0.721142,0.0849714
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9440389294403893, 0.9464027040077257, 0.944...",0.945794,"[0.8247011952191236, 0.6340425531914894, 0.799...",0.705103,0.0633952


In [115]:
Breath_2_dataframe

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,-5.977006,-5.839786,-6.696734,1.304895,-3.281355,-1.167873,-2.562443,-12.080556,-6.502465,1.550612,...,8.564801,14.995866,8.731224,15.448228,15.523052,13.974631,10.353719,14.330583,1.0,2030_P_Asthma_M_after_TBD_Y.json
1,-5.392328,-7.524056,-7.517287,2.365323,-8.092963,-8.338172,-1.899475,-18.203737,-5.927729,-3.604729,...,12.852849,16.923776,9.882452,21.771327,15.493454,16.031112,11.806349,13.461008,1.0,2030_P_Asthma_M_after_TBD_Y.json
2,-6.094472,-6.008680,-3.635710,13.471389,-8.094610,-4.368420,3.522561,-27.064741,1.597966,-11.922719,...,10.690758,10.521691,11.272635,28.299939,9.932172,17.512684,11.181277,10.633954,1.0,2030_P_Asthma_M_after_TBD_Y.json
3,-6.411079,-4.696532,-8.362793,5.046116,-5.932999,-4.862881,-2.530029,-13.778201,-1.507559,-0.511943,...,11.116691,13.005401,11.214902,18.477102,11.523132,12.314678,11.562355,14.261425,1.0,2030_P_Asthma_M_after_TBD_Y.json
4,-6.153268,-2.271515,-6.480360,1.037759,-6.298301,0.371330,-2.558691,-6.554385,-0.536011,7.503430,...,10.132870,11.212203,9.080807,11.090978,10.706592,13.925596,9.745181,18.931210,1.0,2030_P_Asthma_M_after_TBD_Y.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,-2.258854,-20.925547,-30.688681,-11.165256,1.782119,3.605799,8.433943,1.786737,7.831882,10.166328,...,13.158405,10.809771,13.440098,15.748632,16.570694,15.036669,9.858660,20.958058,0.0,1306_C_Na_M_Na_Na_N.json
2302,-2.694701,-20.641974,-30.325957,-4.525265,3.916112,5.415410,6.376050,-4.073862,3.050696,6.971827,...,11.891667,12.133006,10.667039,12.689224,11.860668,12.468933,10.942633,23.800505,0.0,1306_C_Na_M_Na_Na_N.json
2303,-2.970527,-22.292854,-30.628427,-0.327121,1.277205,1.482204,10.595406,-6.064481,-1.246275,10.527364,...,11.157549,11.200076,14.186789,11.707877,9.938035,14.915668,9.559192,22.041033,0.0,1306_C_Na_M_Na_Na_N.json
2304,-3.004799,-22.885229,-31.412037,-3.358953,-2.857134,-0.801861,7.464936,-5.213664,-5.521248,15.240395,...,8.769014,9.863838,11.992603,11.997416,11.934708,18.539998,9.961911,20.337116,0.0,1306_C_Na_M_Na_Na_N.json


In [116]:
Breath_dataframe_25

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,-5.819893,-0.807137,-5.621214,5.254770,-3.315850,5.227212,2.699075,-10.106676,-2.065468,3.210382,...,7.525397,13.206269,8.542131,12.448360,11.166359,14.285654,9.299346,14.721490,1.0,2030_P_Asthma_M_after_TBD_Y.json
1,-4.239213,-5.541188,-4.688828,-0.346142,-6.847378,-2.986326,-0.986394,-12.044204,-6.977862,1.198398,...,12.489347,20.389885,9.178060,16.179534,22.209952,15.749400,12.588714,17.929530,1.0,2030_P_Asthma_M_after_TBD_Y.json
2,-5.354658,-5.668823,-2.405999,10.432677,-7.490145,-3.756268,4.655690,-25.750033,-1.074364,-7.176282,...,10.019886,11.460915,10.751927,26.900385,8.573363,14.580283,11.540474,11.574759,1.0,2030_P_Asthma_M_after_TBD_Y.json
3,-5.092278,-2.529580,-9.688043,9.188193,-12.220343,-6.892942,3.612338,-20.445863,4.298290,-0.158647,...,16.611896,13.117266,10.425394,25.338480,12.605503,16.826520,15.302202,16.827541,1.0,2030_P_Asthma_M_after_TBD_Y.json
4,-5.227036,-2.247611,-6.643047,3.928853,-4.373642,5.126797,0.458337,-8.708336,0.721195,12.353288,...,6.839993,9.704536,6.540969,11.134982,11.737423,17.516899,8.658353,22.654186,1.0,2030_P_Asthma_M_after_TBD_Y.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,-2.151245,-20.350943,-30.374809,-15.213687,1.593645,2.917599,8.487529,2.739748,16.710217,13.032672,...,12.987489,10.466126,12.237796,20.048685,19.411646,18.208179,9.674810,21.100815,0.0,1306_C_Na_M_Na_Na_N.json
2302,-2.248064,-20.340445,-29.144326,-7.463857,8.039509,3.117647,3.994106,2.652726,4.215575,9.145722,...,12.788172,9.728738,10.356971,14.333728,15.189778,13.998471,10.891011,22.383180,0.0,1306_C_Na_M_Na_Na_N.json
2303,-3.028981,-21.319713,-30.583212,-3.390940,4.082062,3.598154,6.138509,-4.684856,3.785632,5.976167,...,13.133627,9.584542,9.770108,10.390636,10.695590,11.538736,10.293202,24.152109,0.0,1306_C_Na_M_Na_Na_N.json
2304,-2.974233,-22.775266,-30.338571,3.460445,1.870983,-2.858343,8.577335,-2.936660,-4.193408,15.106904,...,8.048638,9.262883,11.993682,10.802477,9.773258,17.282774,8.385744,19.409659,0.0,1306_C_Na_M_Na_Na_N.json
